In [1]:
import tensorflow as tf
import h5py as h5
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
VOCAB_SIZE = 12602
MAX_QUESTION_LEN = 26

In [29]:
class VQANet:
    def __init__(self, combine_type, question_embed_dim, lstm_dim, n_answers):
        self.combine_type = combine_type
        self.question_embed_dim = question_embed_dim
        self.lstm_dim = lstm_dim
        self.n_answers = n_answers
        self.build()
        
    def build(self):
        if self.combine_type == 'show-and-tell':
            image_features = tf.keras.layers.Input(shape=(4096,), 
                                                   dtype='float32')
            
            image_embedding = tf.keras.layers.Dense(units=self.question_embed_dim, 
                                                    activation='elu',
                                                    name='image_embedding')(inputs=image_features)

            image_embedding = tf.keras.layers.Reshape((1, self.question_embed_dim))(image_embedding)
            
            question_input = tf.keras.layers.Input(shape=(MAX_QUESTION_LEN,), 
                                                   dtype='int32',
                                                   name='question_input')
            
            question_embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
                                                           output_dim=self.question_embed_dim, 
                                                           input_length=MAX_QUESTION_LEN,
                                                           name='question_embedding')(inputs=question_input)
            
            image_question_embedding = tf.keras.layers.Concatenate(axis=1, 
                                                                   name='image_question_embedding')(inputs=[image_embedding, question_embedding])
            
            question_features, last_h, _ = tf.keras.layers.LSTM(units=self.lstm_dim, 
                                                                return_sequences=True, 
                                                                return_state=True, 
                                                                name='question_generator')(inputs=image_question_embedding)

            question_pred = tf.keras.layers.TimeDistributed(layer=tf.keras.layers.Dense(units=VOCAB_SIZE, 
                                                                  activation='softmax', 
                                                                  name='word_classifier'))(inputs=question_features)
            
            # question_pred[:-1] ignores the last output. Need to add <START> and <END>.
            question_pred = tf.keras.layers.Lambda(lambda x: x[:, :-1, :], 
                                                   name='ignore_last_word')(inputs=question_pred)
            
            answer_fc1 = tf.keras.layers.Dense(units=1000,
                                                activation='elu',
                                                name='answer_dense_1')(inputs=last_h)
            
            answer_pred = tf.keras.layers.Dense(units=self.n_answers,
                                                activation='softmax',
                                                name='answer_classifier')(inputs=answer_fc1)
            
            self.model = tf.keras.Model(inputs=[image_features, question_input], 
                                        outputs=[answer_pred])  
            
            
            optimizer = tf.keras.optimizers.Adam(lr=0.001)
            self.model.compile(loss='categorical_crossentropy', 
                               optimizer=optimizer, metrics=['accuracy'])
    
    def train(self, x_train, y_train, x_val, y_val, batch_size, epochs):
        self.model.fit(x=x_train, 
                       y=y_train, 
                       batch_size=batch_size, 
                       epochs=epochs, 
                       verbose=1,
                       validation_split=0.2,
                       shuffle=True)

In [31]:
batch_size = 32
epochs = 100
question_embed_dim = 256
lstm_dim = 512
n_answers = 1000
n_train = 500

In [32]:
ques = h5.File("../../data/data_train_val/data_prepro.h5", "r")
ques_train = ques['ques_train'][:n_train]
ques_to_image_train = ques['img_pos_train'][:n_train] - 1

ans_train = tf.keras.utils.to_categorical(y=ques['answers'][:n_train],
                                          num_classes=n_answers)

img_feat = h5.File("../../data/data_train_val/data_img.h5", "r")
img_train = np.array(img_feat['images_train'])[ques_to_image_train]

print(img_train.shape)

model = VQANet(combine_type='show-and-tell', 
               question_embed_dim=question_embed_dim, 
               lstm_dim=lstm_dim,
               n_answers=n_answers)

# Issue: ques_train input and output are the same.
model.train(x_train=[img_train, ques_train], 
            y_train=[ans_train], 
            x_val=[], 
            y_val=[], 
            batch_size=batch_size, 
            epochs=epochs)

(500, 4096)
Train on 400 samples, validate on 100 samples
Epoch 1/100
400/400 [==============================] - 17s 43ms/step - loss: 5.6351 - acc: 0.1425 - val_loss: 4.4053 - val_acc: 0.2800
Epoch 2/100
400/400 [==============================] - 8s 19ms/step - loss: 4.0915 - acc: 0.1775 - val_loss: 4.7060 - val_acc: 0.1900
Epoch 3/100
400/400 [==============================] - 7s 18ms/step - loss: 3.8622 - acc: 0.2150 - val_loss: 4.7887 - val_acc: 0.2800
Epoch 4/100
400/400 [==============================] - 7s 18ms/step - loss: 3.8185 - acc: 0.2025 - val_loss: 4.8221 - val_acc: 0.1900
Epoch 5/100
400/400 [==============================] - 7s 18ms/step - loss: 3.7442 - acc: 0.2025 - val_loss: 4.7915 - val_acc: 0.2800
Epoch 6/100
400/400 [==============================] - 7s 18ms/step - loss: 3.7510 - acc: 0.1700 - val_loss: 4.9276 - val_acc: 0.2800
Epoch 7/100
400/400 [==============================] - 7s 18ms/step - loss: 3.7525 - acc: 0.1975 - val_loss: 4.9251 - val_acc: 0.2800
Epo

400/400 [==============================] - 7s 18ms/step - loss: 1.8039 - acc: 0.4325 - val_loss: 6.4851 - val_acc: 0.1700
Epoch 62/100
400/400 [==============================] - 7s 19ms/step - loss: 1.8875 - acc: 0.4075 - val_loss: 5.9505 - val_acc: 0.2200
Epoch 63/100
400/400 [==============================] - 7s 18ms/step - loss: 1.7122 - acc: 0.4600 - val_loss: 6.2806 - val_acc: 0.1900
Epoch 64/100
400/400 [==============================] - 8s 19ms/step - loss: 1.5222 - acc: 0.5275 - val_loss: 6.8882 - val_acc: 0.2900
Epoch 65/100
400/400 [==============================] - 7s 18ms/step - loss: 1.7149 - acc: 0.4400 - val_loss: 6.6844 - val_acc: 0.2000
Epoch 66/100
400/400 [==============================] - 7s 18ms/step - loss: 1.6315 - acc: 0.4950 - val_loss: 6.4104 - val_acc: 0.2200
Epoch 67/100
400/400 [==============================] - 7s 18ms/step - loss: 1.6072 - acc: 0.5375 - val_loss: 6.4825 - val_acc: 0.2400
Epoch 68/100
400/400 [==============================] - 7s 18ms/step